In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
keys = ['采集时间', '水温', 'pH', '溶解氧', '电导率', '浊度', '高锰酸盐指数', '氨氮', '总磷', 
            '总氮', '湿度', '室温', '叶绿素', '藻密度']


In [13]:
from scipy.stats import zscore
# Data value clean
# Methods:
#     Numeric Outlier --> 通过计算第一和第三分位数　小于和大于其二的去掉
#     Z-score --> 计算此数据点和总数据标准差的比，
def detect_outlier(df, label, rate=4):
    dfvals = df[label]
    indices = np.array(list(map(lambda x: not x, np.isnan(dfvals))))
    values = dfvals[indices]
    Q1 = np.percentile(values, rate)
    Q3 = np.percentile(values, 100-rate)    
    
    IQR = Q3 - Q1
    
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR
    
    outlier_list = dfvals[(dfvals < lower_limit) | (dfvals > upper_limit)].index
    return outlier_list

def compute_zscore(df, label, threshold=2.5):
    data = df[label]
    all_values = data.values.copy()
    indices = np.array(list(map(lambda x: not x, np.isnan(all_values))))
    values = all_values[indices]
    z_value = zscore(values)
    all_values[indices] = z_value
    ls = data[np.abs(all_values) > threshold].index
    return ls

def myFilter(df, labels = ['电导率', '高锰酸盐指数'], name='./data_filtered.csv'):
    dfn = df.copy()
    for label in labels:
        dfvals = dfn[label].values.copy()
        out_list = compute_zscore(dfn, label, 1.5)
        dfvals[out_list] = None
        dfn[label] = dfvals
    dfn.to_csv(name)
    return True


data = pd.read_csv('./data.csv')
myFilter(data)
# label = '电导率'
# outs1 = compute_zscore(data, label, 1.5)
# outs2 = detect_outlier(data, label, 4)
# print(outs1)
# print(outs2)
# label = '高锰酸盐指数'
# outs1 = compute_zscore(data, label, 1.5)
# outs2 = detect_outlier(data, label, 4)
# print(outs1)
# print(outs2)

True

In [3]:
# # Data type clean
# original_data = pd.read_excel('./original_data.xls', header=2)
# # print(original_data.keys())
# data = pd.DataFrame()
# ls = [pd.to_datetime(arg=t, format='%Y-%m-%d %H:%M:%S') for t in original_data['采集时间']]
# data['时间'] = ls
# for i in range(1, len(keys)):
#     ls = []
#     for val in original_data[keys[i]]:
#         if val == '--':
#             fval = None
#         else:
#             fval = np.abs(float(val))
#         ls.append(fval)
#     data[keys[i]] = ls
# data.to_csv('data.csv', mode='a')
# values = data['pH']
# values[0]
# values = list(filter(lambda x: x>0, values))